In [115]:
import random

from deap import base
from deap import creator
from deap import tools

import pandas_datareader as pdr
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
yf.pdr_override()

In [116]:
days = 504
end = dt.datetime.today()
start = end - dt.timedelta(days=days)

df_sp500 = yf.download("^GSPC", start=start, end=end)#['Adj Close']

df_sp500.drop(['Open', 'Close', 'High', 'Low'], axis=1, inplace=True)

[*********************100%***********************]  1 of 1 completed


In [117]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

C:\Users\OEM\anaconda3\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\OEM\anaconda3\lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [123]:
toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_int", random.randint, 0, 100)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_int, 3)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [131]:
def eval_fitness(list_xyz):
    #extract x y z from list
    x = list_xyz[0]
    y = list_xyz[1]
    z = list_xyz[2]
    
    #calculate rolling means and add to df
    df_sp500['x_sma'] = df_sp500['Adj Close'].rolling(x).mean()
    df_sp500['y_sma'] = df_sp500['Adj Close'].rolling(y).mean()
    
    #generate BUY/SELL signals and add to df
    for ind in df_sp500.index:
        if df_sp500.loc[ind, 'x_sma'] /  df_sp500.loc[ind, 'y_sma'] > (z/100) :    
            df_sp500.loc[ind, 'Signal'] = 'BUY'

        else:
            df_sp500.loc[ind, 'Signal'] = 'SELL' 

    #Initialise variables
    buy_price = 0 
    sell_price = 0
    bank = 10000
    
    #Loop through df and check entry signal (SELL to BUY) or exit signal (BUY to SELL) generated
    for i in range(len(df_sp500)): 
            
        if df_sp500.iloc[i, 4] == 'BUY' and df_sp500.iloc[i-1, 4] == 'SELL':
            buy_price = df_sp500.iloc[i, 0]
            buy_date = df_sp500.index[i]
            #print('Bought at ${} on {}'.format(buy_price, buy_date))
                  
        elif df_sp500.iloc[i, 4] == 'SELL' and df_sp500.iloc[i-1, 4] == 'BUY' and i > 0:
                sell_price = df_sp500.iloc[i, 0]
                sell_date = df_sp500.index[i]
                #print('Sold at ${} on {}'.format(sell_price, sell_date))
                
        #Check if buy and sell complete and update bank if so        
        if buy_price > 0 and sell_price > 0:
            bank = bank + (sell_price - buy_price)
            buy_price = 0
            sell_price = 0
            #print("New bank = {}".format(bank))
            
    #print("Final bank = {}".format(bank))
    return bank,      

In [138]:
toolbox.register("evaluate", eval_fitness)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, up=5, low=5, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

In [139]:
def main():
    pop = toolbox.population(n=25)
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    CXPB, MUTPB = 0.5, 0.3
    # Extracting all the fitnesses of 
    fits = [ind.fitness.values[0] for ind in pop]
    # Variable keeping track of the number of generations
    g = 0
    print(g)

    # Begin the evolution
    while g < 100:
        # A new generation
        g = g + 1
        print("-- Generation %i --" % g)
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
            
        pop[:] = offspring
        
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]

        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5

        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)

In [140]:
main()

0
-- Generation 1 --
  Min 10000.0
  Max 10405.919921875
  Avg 10053.284814453125
  Std 123.32772459013239
-- Generation 2 --
  Min 9418.279541015625
  Max 10465.490234375
  Avg 10035.833984375
  Std 217.28709713046297
-- Generation 3 --
  Min 10000.0
  Max 10523.289794921875
  Avg 10163.62439453125
  Std 179.6687699967814
-- Generation 4 --
  Min 10000.0
  Max 10523.289794921875
  Avg 10287.66595703125
  Std 212.7892333299027
-- Generation 5 --
  Min 10000.0
  Max 10523.289794921875
  Avg 10395.40673828125
  Std 187.36193647543263
-- Generation 6 --
  Min 10000.0
  Max 10523.289794921875
  Avg 10438.705869140626
  Std 177.8250837662196
-- Generation 7 --
  Min 10000.0
  Max 10523.289794921875
  Avg 10401.466650390625
  Std 213.4821678622468
-- Generation 8 --
  Min 10000.0
  Max 10523.289794921875
  Avg 10479.11462890625
  Std 141.73483352005573
-- Generation 9 --
  Min 10000.0
  Max 10523.289794921875
  Avg 10479.11462890625
  Std 141.73483352005573
-- Generation 10 --
  Min 10000.0
